In [ ]:
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import urljoin
import logging
from pathlib import Path

In [ ]:
!nvidia-smi
!pip install -q transformers datasets accelerate evaluate sentencepiece

import os, json, random
import numpy as np
import torch
from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer, default_data_collator, set_seed
import evaluate

set_seed(42)
device = "cuda" if torch.cuda.is_available() else "cpu"

Thu Oct 23 18:15:54 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   41C    P8             12W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
from transformers import AutoTokenizer
import torch

# Load the tokenizer (assuming it's not already loaded)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def generate_embedding(text):
    """Generates an embedding for a given text chunk."""
    # Tokenize the text
    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=512)

    # Move inputs to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Get the embeddings from the model
    with torch.no_grad():
        outputs = embedding_model(**inputs)

    # Mean pooling to get a single vector for the chunk
    # We need to handle the attention mask to exclude padding tokens
    embeddings = outputs.last_hidden_state
    mask = inputs['attention_mask'].unsqueeze(-1).expand(embeddings.size()).float()
    sum_embeddings = torch.sum(embeddings * mask, 1)
    sum_mask = torch.clamp(mask.sum(1), min=1e-9)
    mean_pooled_embeddings = sum_embeddings / sum_mask

    return mean_pooled_embeddings[0].cpu().numpy() # Return as numpy array on CPU

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
def add_embeddings(examples):
    """Applies the embedding function to each chunk in a batch."""
    all_embeddings = []
    for chunk_list in examples['chunks']:
        chunk_embeddings = [generate_embedding(chunk) for chunk in chunk_list]
        all_embeddings.append(chunk_embeddings)
    return {'embeddings': all_embeddings}

# Apply the function to the dataset
loaded_dataset = loaded_dataset.map(add_embeddings, batched=True)

print(loaded_dataset)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'filename', 'chunks', 'embeddings'],
    num_rows: 9
})


In [ ]:
dataset_with_embeddings_save_path = "aaib_dataset_with_embeddings"
loaded_dataset.save_to_disk(dataset_with_embeddings_save_path)
print(f"Dataset with embeddings saved successfully to {dataset_with_embeddings_save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset with embeddings saved successfully to aaib_dataset_with_embeddings


In [ ]:
!pip install PyMuPDF

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 116.0 MB/s eta 0:00:00


## Extraer texto de pdfs


In [ ]:
import fitz
import os

pdf_folder_path = 'aaib_reports'
extracted_texts = []
filenames = []

for filename in os.listdir(pdf_folder_path):
    if filename.endswith('.pdf'):
        file_path = os.path.join(pdf_folder_path, filename)
        try:
            document = fitz.open(file_path)
            text = ''
            for page_num in range(document.page_count):
                page = document.load_page(page_num)
                text += page.get_text()
            extracted_texts.append(text)
            filenames.append(filename)
            document.close()
        except Exception as e:
            print(f"Error processing {filename}: {e}")

print(f"Extracted text from {len(extracted_texts)} PDF files.")

FileNotFoundError: [Errno 2] No such file or directory: 'aaib_reports'

## Estructurar los datos



In [ ]:
data = {'text': extracted_texts, 'filename': filenames}
print(data.keys())

dict_keys(['text', 'filename'])


## Crear un dataset de hugging face


In [ ]:
from datasets import Dataset

hf_dataset = Dataset.from_dict(data)
print(hf_dataset)

Dataset({
    features: ['text', 'filename'],
    num_rows: 9
})


## Guardar el dataset



In [ ]:
dataset_save_path = "aaib_dataset"
hf_dataset.save_to_disk(dataset_save_path)
print(f"Dataset saved successfully to {dataset_save_path}")

Saving the dataset (0/1 shards):   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset saved successfully to aaib_dataset


## Cargar el dataset

Fue guardado previamente desde el directorio "aaib_dataset".


In [ ]:
from datasets import load_from_disk

loaded_dataset = load_from_disk("aaib_dataset")
print(loaded_dataset)

Dataset({
    features: ['text', 'filename'],
    num_rows: 9
})


## Definir estrategia de "chunking"



In [ ]:
def chunk_text_by_size(text, chunk_size=255):
    """Splits text into chunks of approximately a fixed size."""
    chunks = []
    current_chunk = ""
    words = text.split()
    for word in words:
        if len(current_chunk) + len(word) + 1 <= chunk_size:
            if current_chunk:
                current_chunk += " " + word
            else:
                current_chunk += word
        else:
            chunks.append(current_chunk)
            current_chunk = word
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

## Aplicar "chunking" a los datos


In [ ]:
def apply_chunking(example):
    """Applies the chunking function to the text in an example."""
    text = example['text']
    chunks = chunk_text_by_size(text, chunk_size=320) # Use the new function and size
    return {'chunks': chunks}

loaded_dataset = loaded_dataset.map(apply_chunking)
print(loaded_dataset)

Map:   0%|          | 0/9 [00:00<?, ? examples/s]

Dataset({
    features: ['text', 'filename', 'chunks'],
    num_rows: 9
})


## Indexar los "chunks"


Crear un índice que permita acceder fácilmente a los los "chunks".


In [ ]:
document_index = {}

for example in loaded_dataset:
    filename = example['filename']
    chunks = example['chunks']
    document_index[filename] = chunks

print(f"Indexed {len(document_index)} files.")

Indexed 9 files.


## Mostrar el primer "chunk" de los primeros 3 pdfs




In [ ]:
for i in range(min(3, len(loaded_dataset))):
    example = loaded_dataset[i]
    filename = example['filename']
    chunks = example['chunks']
    if chunks:
        first_chunk = chunks[0]
        first_chunk_size = len(first_chunk)
        print(f"Filename: {filename}")
        print(f"First Chunk: {first_chunk}")
        print(f"Size of First Chunk: {first_chunk_size} characters")
        print("-" * 20)
    else:
        print(f"Filename: {filename}")
        print("No chunks found for this document.")
        print("-" * 20)

Filename: Robinson_R22_Beta_G-KKRN_09-25.pdf
First Chunk: 6 All times are UTC © Crown copyright 2025 AAIB Bulletin: 9/2025 G-KKRN AAIB-30658 Accident Aircraft Type and Registration: Robinson R22 Beta, G-KKRN No & Type of Engines: 1 Lycoming O-320-B2C piston engine Year of Manufacture: 1989 (Serial no: 1201) Date & Time (UTC): 3 February 2025 at 1700 hrs Location: Near
Size of First Chunk: 312 characters
--------------------
Filename: Boeing_737-8K5_G-TAWB_11-25.pdf
First Chunk: 1 All times are UTC © Crown copyright 2025 AAIB Bulletin: G-TAWB AAIB-30560 Accident Aircraft Type and Registration: Boeing 737-8K5, G-TAWB No & Type of Engines: 2 CFM56-7B27E turbofan engines Year of Manufacture: 2012 (Serial no: 37242) Date & Time (UTC): 16 December 2024 at 1628 hrs Location: East Midlands Airport
Size of First Chunk: 317 characters
--------------------
Filename: Piper_PA_46-350P_N3117J_09-25.pdf
First Chunk: 4 All times are UTC © Crown copyright 2025 AAIB Bulletin: 9/2025 N3117J AAIB-30893 Ac

## Cargar un modelo de embeddings



In [ ]:
from transformers import AutoModel

model_name = 'sentence-transformers/all-MiniLM-L6-v2'
embedding_model = AutoModel.from_pretrained(model_name).to(device)
print(f"Model {model_name} loaded successfully and moved to {device}.")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

Model sentence-transformers/all-MiniLM-L6-v2 loaded successfully and moved to cuda.
